In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import collections
import os 

import warnings
warnings.filterwarnings("ignore")

import xesmf as xe

from utils import (_remove_leap_days, compute_daily_climo, calculate_scaling_factor, apply_scale_factor, 
                   _convert_ds_longitude)
from regridding import apply_weights

import dask.distributed as dd
import dask_kubernetes as dk
import dask
import rhg_compute_tools.kubernetes as rhgk

### This notebook is a test of all the steps for Spatial Disaggregation to get a handle on the total CPU time it will take for this part of BCSD. 

Once-off steps: 

1. compute multi-decade daily climatologies of ERA-5 at obs-res and coarsen it to model-res (they, e.g. NASA-NEX, do not say how, we will do bilinear for consistency with later step)

Per model/scenario/experiment steps:

1. subtract (or divide for precip) BC’ed model data at model-res from obs climo at model resolution to calculate a “scaling factor” 
2. bilinearly interpolate “scaling factor” (using xESMF) from the model grid to the obs grid 
3. Apply scaling factor by adding (for temp) and multiplying (for precip) the “scaling factor” to the obs-res daily climatology 

NOTE: For the purpose of being conservative with timing, the "coarsen obs climatology step to model-res" is in the per model/scenario/experiment step, since there is a wide variety of grids among CMIP6 models. 

In [2]:
client, cluster = rhgk.get_standard_cluster()
cluster

load test bias corrected output from global bias correction prototype notebook (BC'ed NASA GISS CMIP6 data)

In [3]:
workdir = '/gcs/rhg-data/climate/downscaled/workdir'

In [4]:
year = 1990

In [5]:
tmax_model = xr.open_dataset(os.path.join(workdir, 
                                          'global_bias_correction_scaling_test.nc'))

In [6]:
tmax_model = tmax_model.loc[dict(time=slice("%s-01-01" %str(year), "%s-12-31" %str(year)))]

In [7]:
tmax_obs = xr.open_dataset(os.path.join('/gcs/rhg-data/climate/source_data/GMFD/tmax', 
                                         'tmax_0p25_daily_1990-1990.nc')).squeeze(drop=True
                                          ).rename({'latitude': 'lat', 'longitude': 'lon'})

In [8]:
# standardize longitudes 
tmax_obs = _convert_ds_longitude(tmax_obs, lon_name='lon')

Remove leap days from obs 

In [9]:
# remove leap days 
tmax_obs = _remove_leap_days(tmax_obs)

Load daily obs climatology 

In [10]:
climo_obs_fine = (xr.open_dataset(os.path.join(workdir, 'gmfd_test_climo.nc'))
                 .rename({'latitude': 'lat', 'longitude': 'lon'}))

### Interpolate obs climo: fine -> coarse 

In [11]:
%%time 
obs_to_mod_weights = workdir + '/obs_to_mod_bilinear_spatial_disagg.nc'
regridder_obs_to_mod = xe.Regridder(tmax_obs.isel(time=0), tmax_model.isel(time=0), 
                         'bilinear', filename=obs_to_mod_weights, reuse_weights=True)

Reuse existing file: /gcs/rhg-data/climate/downscaled/workdir/obs_to_mod_bilinear_spatial_disagg.nc
CPU times: user 23.9 ms, sys: 16.5 ms, total: 40.4 ms
Wall time: 65.7 ms


In [12]:
%%time
climo_obs_coarse_lazy = xr.map_blocks(apply_weights, regridder_obs_to_mod, 
                                args=[climo_obs_fine['tmax']])

CPU times: user 450 ms, sys: 867 ms, total: 1.32 s
Wall time: 1.24 s


In [13]:
%%time 
climo_obs_coarse = climo_obs_coarse_lazy.compute()

CPU times: user 425 µs, sys: 0 ns, total: 425 µs
Wall time: 434 µs


### Compute scaling factor by subtracting for temperature, dividing for precip, the BC'ed model data at model-res from obs climo at model-res. 

In [14]:
chunks = {'lat': 75, 'lon': 75}
climo_obs_coarse = climo_obs_coarse.chunk(chunks)

In [15]:
%%time 
scale_factor_coarse = xr.map_blocks(calculate_scaling_factor, 
                                    tmax_model, args=[climo_obs_coarse, 'tasmax'])
sfc = scale_factor_coarse.compute()

CPU times: user 15.9 s, sys: 799 ms, total: 16.7 s
Wall time: 16.5 s


### Interpolate scaling factor: coarse (model grid) -> fine (obs grid)

In [16]:
%%time
mod_to_obs_weights = workdir + '/mod_to_obs_bilinear_spatial_disagg.nc'
regridder_mod_to_obs = xe.Regridder(tmax_model.isel(time=0), 
                                    tmax_obs.isel(time=0), 
                         'bilinear', filename=mod_to_obs_weights, reuse_weights=True)

Reuse existing file: /gcs/rhg-data/climate/downscaled/workdir/mod_to_obs_bilinear_spatial_disagg.nc
CPU times: user 49.3 ms, sys: 76.8 ms, total: 126 ms
Wall time: 153 ms


In [17]:
%%time
sfc = sfc.drop('dayofyear')
sff_lazy = xr.map_blocks(apply_weights, regridder_mod_to_obs, 
                                args=[sfc])
sff_lazy_compute = sff_lazy.compute()

CPU times: user 4.43 s, sys: 3.65 s, total: 8.07 s
Wall time: 7.56 s


### Add (or multiply for precip) the scaling factor to the obs-res daily climatology

In [18]:
sff_ds = sff_lazy_compute.to_dataset(name='scale_factor_fine')

In [19]:
mod_yr_downscaled = apply_scale_factor(sff_ds['scale_factor_fine'], climo_obs_fine['tmax'], 
                                           sff_ds.time.dt.dayofyear)
filepath = workdir + '/spatial_disagg_prototype/%s.nc' %str(year)
mod_yr_downscaled.to_netcdf(filepath)
print("finished %s" %str(year))

finished 1990
